# Importing and reading the data file and performing EDA

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("training.csv", encoding='ISO-8859-1') #encoding because it gave some error

In [3]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [28]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer #to feed the numerical type data to model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
column_names=['target','id','date','flag','user','text']
df=pd.read_csv("training.csv",names=column_names,encoding='ISO-8859-1')

In [7]:
df.columns

Index(['target', 'id', 'date', 'flag', 'user', 'text'], dtype='object')

In [8]:
 df.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

"__TWEETS ARE DISTRIBUTED EQUALLY__"

In [9]:
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

"__CONVERTING 4 TO 1 (DATA PREPROCESSING)__"

In [10]:
df.replace({'target':{4:1}},inplace=True)

In [11]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

**_zero means negetive tweet and one means positive tweet_**

# 1.Stemming/ Lemmatization

In [12]:
port_stem=PorterStemmer()

In [13]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content= stemmed_content.lower()
    stemmed_content= stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #port_stem will basically reduce the word to keyword
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [14]:
df['stemmed_content']=df['text'].apply(stemming)

In [17]:
print(df['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [18]:
print(df['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [59]:
x=df['stemmed_content'].values
y=df['target'].values

In [60]:
xt,xtt,yt,ytt=train_test_split(x,y,test_size=0.2,random_state=2)

# CONVERTING THE DATA TO INTEGER (VECTORIZING)

In [61]:
vectorizer=TfidfVectorizer()

In [62]:
xt=vectorizer.fit_transform(xt)
xtt=vectorizer.transform(xtt)

In [63]:
lr=LogisticRegression()

In [64]:
lr.fit(xt,yt)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [65]:
pred=lr.predict(xt)
accuracy=accuracy_score(yt,pred)

In [66]:
print("accuracy is: ",accuracy)

accuracy is:  0.8023890625


_**NOW PREDICTING AND CHECKING WITH THE TEST DATA**_

In [67]:
predick=lr.predict(xtt)
accuracys=accuracy_score(ytt,predick)

In [69]:
print("accuracy is: ",accuracys)

accuracy is:  0.77656875


**MODEL ACCURACY IS 77.65%**

In [72]:
test=["very bad experience no good services"]
test=vectorizer.transform(test)
tes=lr.predict(test)

In [73]:
print(tes)

[0]


**as it comes out to be zero which means our model is working ok, so lets make a _function_ to do the tasks**

In [75]:
def check(test):
    test=vectorizer.transform(test)
    tes=lr.predict(test)
    if(tes==0):
        print("The tweet is negative")
    if(tes==1):
        print("The tweet is positive")

In [77]:
test=["WOW! Such a great service and taasty food, really happy with the aste of biriyani"]
check(test)

The tweet is positive


__LETS CHECK WITH SOMETHIGNCONFUSING WITH BOTH POSITIVE AND NEGATIVE REVIEW__

In [79]:
test=["The government is doing good work but the corruption is on rise and developement is very less but still our economy is growing at good rate"]
check(test)

The tweet is negative
